In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [2]:
from pynq.lib import Pmod_ADC, Pmod_DAC
import time
import ctypes
import numpy as np
import array
from pynq import allocate
import numpy as np 


In [3]:
dac = Pmod_DAC(base.PMODB)

In [7]:
def run_transmit():
    # Load transmit data from file
    with open("/home/xilinx/pynq/tx_out.dat", mode='rb') as file:
            file_content = file.read()
    float_strs = file_content.split(b" ")
    
    volt_floats = [float(x) for x in float_strs if x != b'']
    
    max_voltage = max([abs(x) for x in volt_floats])
    
    # shift the voltages so all are positive and scale to 12 bit uint (store as 16 bit uint)
    volt_pos = [x + max_voltage for x in volt_floats]
    #DAC_MAX = 4095
    DAC_MAX = 1600
    #DAC_MAX = 1310
    #DAC_MAX = 2457
    #DAC_MAX = 491
    
    volt_scaled = np.asarray([np.uint16(x*(DAC_MAX/(2*max_voltage))) for x in volt_pos])
    
    length = len(volt_scaled)
    py_buffer = allocate(shape=(length,), dtype=np.int32)
    
    for i in range(length):
        py_buffer[i] = volt_scaled[i]
        
    i = 1
    #while i < 10:
    while True:
        dac.write(py_buffer.physical_address|0x00000003)
        #i = i+1
    
    return py_buffer


In [8]:
run_transmit()

KeyboardInterrupt: 

In [ ]:
## Square Wave ##

In [ ]:
def run_transmit_sq():
        
    length = 3776
    py_buffer = allocate(shape=(length,), dtype=np.int32)
    
    a = np.zeros((1888))
    b = np.full(1888, 1600)

    py_buffer[::2] = a
    py_buffer[1::2] = b

    #print(hex(pointer))
    #print((pointer))
    #for i in range(len(volt_scaled)):
    #    print(hex(ctypes.addressof(volt_scaled.buffer_info()[0].contents[i])))
    #    print(volt_scaled[i].data)
    
    #print("writing to dac...")
    while True:
        dac.write(py_buffer.physical_address|0x00000003)
    
    return py_buffer

In [ ]:
run_transmit_sq()

In [ ]:
## Sine Wave ##

In [ ]:
def run_transmit_sine():
    
    
    a = [32, 48, 55, 60, 64, 60, 55, 48, 32, 16, 9, 5, 9, 16, 32]
    
    length = len(a)

    py_buffer = allocate(shape=(length,), dtype=np.int32)

    for i in range(length):
        py_buffer[i] = a[i] * 20;
        
        #print("writing to dac...")
        
    while True:
        dac.write(py_buffer.physical_address|0x00000003)
    
    return py_buffer

In [ ]:
run_transmit_sine()

In [17]:
def run_transmit_dc(value):
        
    length = 3776
    py_buffer = allocate(shape=(length,), dtype=np.int32)
    
    digital = value/0.0006105
    
    
    data = np.full(3776, digital) 
    
    for i in range(3776):
        py_buffer[i] = data[i];

    while True:
        dac.write(py_buffer.physical_address|0x00000003)
    


In [36]:
run_transmit_dc(1.0)

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt 

tx = run_transmit_sq()
plt.plot(tx)
plt.show

In [ ]:
import matplotlib.pyplot as plt 

tx = run_transmit()
plt.plot(tx)
plt.show

In [ ]:
import matplotlib.pyplot as plt 

tx = run_transmit_sine()
plt.plot(tx)
plt.show